### **Data Description**

**Imports**

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split

**Load raw dataset**

In [21]:
dataset = pd.read_csv("broward_data.csv")

**Check data for any glaring issues**

In [43]:
### MISSING DATA ###
# is any data missing? 
missing_vals = dataset.isna().sum().sum()           #num missing values
total_vals = dataset.shape[0] * dataset.shape[1]    #total num values
pct_missing = (missing_vals / total_vals) * 100     #percent of the data missing
print(f"Percentage of the data missing: {pct_missing}%\n")

### DUPLICATES ###
#if any person is in the dataset twice, the duplicate will be removed
dataset.drop_duplicates(subset=[dataset.columns[0]], inplace=True)

### INCORRECT DATA TYPES ###
#make sure these look correct
data_types = dataset.dtypes
#print(data_types)

Percentage of the data missing: 0.0%



**Preliminary Analysis**

In [15]:
### HISTOGRAMS ###


### SCATTER PLOTS ###


### ROC CURVES ###


### DIMENSIONALITY REDUCTION ###


### CLUSTERING ALGORITHMS ###

**Split into test and training**

In [16]:
### IDENTIFY FEATURES AND TARGETS ###

features = ["person_id","screening_date","sex","race","age_at_current_charge",
            "age_at_first_charge","p_charges","p_incarceration","p_probation",
            "p_juv_fel_count","p_felprop_viol","p_murder","p_felassault",
            "p_misdeassault","p_famviol","p_sex_offense","p_weapon","p_fta_two_year",
            "p_fta_two_year_plus","current_violence","current_violence20",
            "p_pending_charge","p_felony","p_misdemeanor","p_violence",
            "total_convictions","p_arrest","p_property","p_traffic","p_drug","p_dui",
            "p_domestic","p_stalking","p_voyeurism","p_fraud","p_stealing","p_trespass",
            "six_month","one_year","three_year","five_year"]
targets = ["general_two_year","general_six_month","drug_two_year","property_two_year",
           "misdemeanor_two_year","felony_two_year","violent_two_year","drug_six_month",
           "property_six_month","misdemeanor_six_month","felony_six_month",
           "violent_six_month"]

X = broward_dataset[features]   # feature values for all rows in raw dataset
y = broward_dataset[targets]    # target values for all rows in raw dataset


### SPLIT INTO TRAIN AND TEST SETS ###
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Normalize/Standardize?**